In [1]:
! pip install semantic-kernel -U

In [2]:
import os
import sys

import semantic_kernel as sk

In [3]:

kernel = sk.Kernel()

In [4]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [5]:
service_id = None

In [6]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_service(skaoai.AzureChatCompletion(deployment_name=deployment,endpoint=endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [7]:
from semantic_kernel.planners.basic_planner import BasicPlanner

In [8]:

planner = BasicPlanner(service_id)

In [9]:
base_skills_directory = '../../../plugins'

In [10]:
import APIPlugin.CustomAPIPlugin as custom_plugin

In [11]:

custom_plugin = kernel.import_plugin_from_object(custom_plugin.CustomAPIPlugin(), plugin_name="CustomAPI")  #.import_native_skill_from_directory(base_plugin , "APIPlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_plugin_from_prompt_directory(base_skills_directory, "TranslatePlugin")

In [12]:
ask = """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate it to Chinese"""

In [13]:
service_id

In [14]:

plan = await planner.create_plan(ask, kernel)

In [15]:
print(plan.generated_plan)

{
    "input": "Travel tips based on current weather in Guangzhou",
    "subtasks": [
        {"function": "CustomAPI.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips", "args": {"input": "weather in Guangzhou"}},
        {"function": "EmailPlugin.WeatherMail", "args": {"input": "weather in Guangzhou tips"}},
        {"function": "TranslatePlugin.Basic", "args": {"language": "Chinese"}}
    ]
}


In [16]:
results = await planner.execute_plan(plan, kernel)

In [17]:
results

"Dear Team,\n\nI hope this email finds you well. As we are preparing for our upcoming business trip to Guangzhou, I wanted to share some important weather and dressing tips to ensure we are all comfortable and prepared for the local conditions.\n\nGuangzhou has a subtropical climate, which means the weather can be quite unpredictable. It is essential that we stay updated with the latest weather forecasts and adjust our plans accordingly. I recommend downloading a reliable weather app on your smartphones for real-time updates.\n\nIn terms of dressing, it is advisable to wear light, breathable clothing due to the city's generally humid conditions. However, it's also important to carry a light jacket or sweater as indoor areas can often be heavily air-conditioned. Don't forget to pack an umbrella or raincoat as well, as sudden showers are common.\n\nFor those planning outdoor activities or sightseeing, please remember to apply sunscreen and wear a hat or sunglasses to protect yourself fro